
# <font color='darkblue'> MODELO 2: BOSQUE ALEATORIO (100 ÁRBOLES) </font>

# <font color='darkblue'> --------------------.--------------------.--------------------.--------------------.--------------------.--------------------. </font>

# <font color='blue'> A. Leer los Datos y hacer últimos Ajustes </font>

### 1. Cargar datos

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# Credenciales

datos_modelo = pd.read_csv(body)

In [2]:
datos_modelo.shape

(8685950, 39)

In [3]:
datos_modelo.head(5)

,tiendaID,marcaID,skuID,gammaID,fechaID,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
0,1,2,1,1,0,0,0.0,70,1.044776,355,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
1,1,2,1,1,1,0,0.0,79,1.128571,379,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,2,1,1,2,0,0.0,87,1.060976,264,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
3,1,2,1,1,3,0,0.0,75,1.041667,136,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
4,1,2,1,1,4,0,0.0,77,1.026667,86,...,0,0.0,79,1.128571,379,1.046961,92120,1.137986,1,1.0


### 2. Quitar NA del último mes por que es información que no se va a utilizar (no se tiene información del mes de abril)

In [4]:
datos_modelo = datos_modelo[datos_modelo.fechaID != 9]

In [5]:
datos_modelo.shape

(7817355, 39)

### 3. Quitar columnas de gammaID y marcaID porque son características que el SKUID trae implícitas

In [6]:
datos_modelo = datos_modelo.drop(['marcaID', 'gammaID'], axis=1)

In [7]:
datos_modelo.shape

(7817355, 37)

# <font color='darkblue'> --------------------.--------------------.--------------------.--------------------.--------------------.--------------------. </font>

# <font color='blue'> B. Hacer Divisiones para Cross Validation (8 Conjuntos de Entrenamiento - 8 Conjuntos de Prueba) </font>

In [8]:
#División periódica
mes = datos_modelo.fechaID.unique() #solo considera 9 periodos de tiempo
mes

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [9]:
#ejemplo
mes[2]

2

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 1. Primera división: 
#### Entrenamiento: fechaID=0 (mes junio) - y: ventas julio
#### Prueba: fechaID=1 (mes julio) - y:ventas agosto

##### 1. Extraer de los datos los conjuntos de entrenamiento y prueba 

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junio_para_predecir_julio = mes[0] #Con infomación de junio quiero predecir julio (entrenamiento)
prueba_julio_para_predecir_en_agosto = mes[1] #Con el modelo anterior quiero predecir agosto (con información de julio)

print('Entrenar con el mes de Junio cuyo índice es:', entrenamiento_junio_para_predecir_julio)
print('Probar con el mes de Julio cuyo índice es:', prueba_julio_para_predecir_en_agosto)

Entrenar con el mes de Junio cuyo índice es: 0
Probar con el mes de Julio cuyo índice es: 1


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_1 = datos_modelo[datos_modelo['fechaID']==entrenamiento_junio_para_predecir_julio]
#entrenamiento_1.head(2) #tiene puros valores de 0 en fechaID
entrenamiento_1.shape #(868595, 37)

(868595, 37)

In [12]:
prueba_1 = datos_modelo[datos_modelo['fechaID']==prueba_julio_para_predecir_en_agosto]
#prueba_1.head(2) #tiene puros valores de 1 en fecha ID
prueba_1.shape #(868595, 37)

(868595, 37)

#### 2. Generar índices 

In [13]:
entrenamiento_1.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_1.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,0,0,0.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0,...,1,1.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0


In [14]:
prueba_1.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_1.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,0.0,79,1.128571,379,1.046961,92120,1.137986,1,1.0,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,1,0,0.0,24,1.043478,379,1.046961,92120,1.137986,1,1.0,...,1,1.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0


#### 3. Dividir en variables de entrada y de salida en cada conjunto

In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_1 = entrenamiento_1.iloc[:, entrenamiento_1.columns != 'variable_y_ventas_del_siguiente_mes']
y_1 = entrenamiento_1.iloc[:, entrenamiento_1.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir en x y y el conjunyo de prueba
prueba_x_1= prueba_1.iloc[:, prueba_1.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_1= prueba_1.iloc[:, prueba_1.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 2. Segunda división: 
#### Entrenamiento: fechaID=0 y 1 (mes junio y julio) - y: ventas Agosto
#### Prueba: fechaID=2 (mes Agosto) - y:ventas septiembre

In [37]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjul_para_predecir_agosto = mes[1] #Con infomación de junio y julio quiero predecir agogsto (entrenamiento)
prueba_agosto_para_predecir_en_septiembre = mes[2] #Con el modelo anterior quiero predecir septiembre (con información de agosto

print('Entrenar con los meses de Junio y Julio cuyos índices son:', "0",entrenamiento_junjul_para_predecir_agosto)
print('Probar con el mes de Agosto cuyo índice es:', prueba_agosto_para_predecir_en_septiembre)

Entrenar con los meses de Junio y Julio cuyos índices son: 0 1
Probar con el mes de Agosto cuyo índice es: 2


In [38]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_2 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjul_para_predecir_agosto]
#entrenamiento_2.head(2)
entrenamiento_2.shape

(1737190, 37)

In [39]:
prueba_2 = datos_modelo[datos_modelo['fechaID']==prueba_agosto_para_predecir_en_septiembre]
#prueba_2.head(2)
prueba_2.shape

(868595, 37)

In [40]:
entrenamiento_2.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_2.head(4)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
2        1     0                     0                                  0.0   
               1                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
2        1     0                                       14   
               1                                       24   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
2        1     0                                  1.076923   
               1                                  1.043478   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
2        1     0                                     355   
               1                                     379   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
2        1     0                                 1.047198   
               1                                 1.046961   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
2        1     0                                   86507   
               1                                   92120   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
2        1     0                                 1.161542   
               1                                 1.137986   

                        ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                   
1        1     0                                     1   
               1                                     1   
2        1     0                                     1   
               1                                     1   

                        ventas_promedio_por_sku_por_mes  ...  \
tiendaID skuID fechaID                                   ...   
1        1     0                                    1.0  ...   
               1                                    1.0  ...   
2        1     0                                    1.0  ...   
               1                                    1.0  ...   

                        lag2mes_ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                           
1        1     0                                             1   
               1                                             1   
2        1     0                                             1   
               1                                             1   

                        lag2mes_ventas_promedio_por_sku_por_mes  \
tiendaID skuID fechaID                                            
1        1     0                                            1.0   
               1     

In [41]:
prueba_2.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_2.head(4)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,2,0,0.0,87,1.060976,264,1.068826,84129,1.12627,0,0.0,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,2,0,0.0,13,1.083333,264,1.068826,84129,1.12627,0,0.0,...,1,1.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0
3,1,2,0,0.0,36,1.028571,264,1.068826,84129,1.12627,0,0.0,...,1,1.0,24,1.000000,355,1.047198,86507,1.161542,1,1.0
4,1,2,0,0.0,0,0.000000,264,1.068826,84129,1.12627,0,0.0,...,1,1.0,0,0.000000,355,1.047198,86507,1.161542,1,1.0


In [42]:
#Dividir x y y del conjunto de entrenamiento 
x_2 = entrenamiento_2.iloc[:, entrenamiento_2.columns != 'variable_y_ventas_del_siguiente_mes']
y_2 = entrenamiento_2.iloc[:, entrenamiento_2.columns == 'variable_y_ventas_del_siguiente_mes']

In [43]:
#Dividir en x y y el conjunyo de prueba
prueba_x_2= prueba_2.iloc[:, prueba_2.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_2= prueba_2.iloc[:, prueba_2.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 3. Tercera división: 
#### Entrenamiento: fechaID=0,1,2(mes junio julio agosto) - y: ventas septiembre
#### Prueba: fechaID=3 (mes septiembre) - y:ventas octubre

In [62]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulago_para_predecir_septiembre = mes[2] 
prueba_septiembre_para_predecir_en_octubre = mes[3] 

print('Entrenar con los meses de Junio,Julio y agosto cuyos índices son:', "0","1",entrenamiento_junjulago_para_predecir_septiembre)
print('Probar con el mes de septiembre cuyo índice es:', prueba_septiembre_para_predecir_en_octubre)

Entrenar con los meses de Junio,Julio y agosto cuyos índices son: 0 1 2
Probar con el mes de septiembre cuyo índice es: 3


In [63]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_3 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulago_para_predecir_septiembre]
#entrenamiento_3.head(2)
entrenamiento_3.shape

(2605785, 37)

In [64]:
prueba_3 = datos_modelo[datos_modelo['fechaID']==prueba_septiembre_para_predecir_en_octubre]
#prueba_3.head(2)
prueba_3.shape

(868595, 37)

In [65]:
entrenamiento_3.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_3.head(3)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
               2                                 1.126270   

                        ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                   
1        1     0                                     1   
               1                                     1   
               2                                     0   

                        ventas_promedio_por_sku_por_mes  ...  \
tiendaID skuID fechaID                                   ...   
1        1     0                                    1.0  ...   
               1                                    1.0  ...   
               2                                    0.0  ...   

                        lag2mes_ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                           
1        1     0                                             1   
               1                                             1   
               2                                             1   

                        lag2mes_ventas_promedio_por_sku_por_mes  \
tiendaID skuID fechaID                                            
1        1     0                                            1.0   
               1                                            1.0   
               2                                            1.0   

                        lag3mes_ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                              
1        1     0                                               70   
               1                                               70   
               2                                               70   

                        lag3mes_ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                               
1        1     0                

In [66]:
prueba_3.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_3.head(3)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,3,0,0.0,75,1.041667,136,1.079365,73612,1.110521,0,0.0,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,3,0,0.0,22,1.047619,136,1.079365,73612,1.110521,0,0.0,...,1,1.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0
3,1,3,0,0.0,32,1.000000,136,1.079365,73612,1.110521,0,0.0,...,1,1.0,24,1.000000,355,1.047198,86507,1.161542,1,1.0


In [67]:
#Dividir x y y del conjunto de entrenamiento 
x_3 = entrenamiento_3.iloc[:, entrenamiento_3.columns != 'variable_y_ventas_del_siguiente_mes']
y_3 = entrenamiento_3.iloc[:, entrenamiento_3.columns == 'variable_y_ventas_del_siguiente_mes']

In [68]:
#Dividir en x y y el conjunyo de prueba
prueba_x_3= prueba_3.iloc[:, prueba_3.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_3= prueba_3.iloc[:, prueba_3.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 4. Cuarta división: 
#### Entrenamiento: fechaID=0,1,2,3 (mes junio julio agosto septiembre) - y: ventas octubre
#### Prueba: fechaID=4 (mes octubre) - y:ventas noviembre

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulagosep_para_predecir_octubre = mes[3]
prueba_octubre_para_predecir_en_noviembre = mes[4]

print('Entrenar con los meses de Junio,Julio,Agosto y Septiembre cuyos índices son:', "0","1","2",entrenamiento_junjulagosep_para_predecir_octubre)
print('Probar con el mes de octubre cuyo índice es:', prueba_octubre_para_predecir_en_noviembre)

Entrenar con los meses de Junio,Julio,Agosto y Septiembre cuyos índices son: 0 1 2 3
Probar con el mes de octubre cuyo índice es: 4


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_4 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulagosep_para_predecir_octubre]
#entrenamiento_4.head(2)
entrenamiento_4.shape

(3474380, 37)

In [12]:
prueba_4 = datos_modelo[datos_modelo['fechaID']==prueba_octubre_para_predecir_en_noviembre]
#prueba_4.head(2)
prueba_4.shape

(868595, 37)

In [13]:
entrenamiento_4.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_4.head(4)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   
               3                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   
               3                                       75   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   
               3                                  1.041667   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   
               3                                     136   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   
               3                                 1.079365   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   
               3                                   73612   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
               2                                 1.126270   
               3                                 1.110521   

                        ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                   
1        1     0                                     1   
               1                                     1   
               2                                     0   
               3                                     0   

                        ventas_promedio_por_sku_por_mes  ...  \
tiendaID skuID fechaID                                   ...   
1        1     0                                    1.0  ...   
               1                                    1.0  ...   
               2                                    0.0  ...   
               3                                    0.0  ...   

                        lag2mes_ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                           
1        1     0                                             1   
               1                                             1   
               2                                             1   
               3                                             1   

                        lag2mes_ventas_promedio_por_sku_por_mes  \
tiendaID skuID fechaID                                            
1        1     0                                            1.0   
               1     

In [14]:
prueba_4.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_4.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,4,0,0.0,77,1.026667,86,1.088608,67113,1.103143,0,0.0,...,0,0.0,79,1.128571,379,1.046961,92120,1.137986,1,1.0
2,1,4,0,0.0,9,1.125000,86,1.088608,67113,1.103143,0,0.0,...,0,0.0,24,1.043478,379,1.046961,92120,1.137986,1,1.0


In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_4 = entrenamiento_4.iloc[:, entrenamiento_4.columns != 'variable_y_ventas_del_siguiente_mes']
y_4 = entrenamiento_4.iloc[:, entrenamiento_4.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir en x y y el conjunyo de prueba
prueba_x_4= prueba_4.iloc[:, prueba_4.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_4= prueba_4.iloc[:, prueba_4.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 5. Quinta división: 
#### Entrenamiento: fechaID=0,1,2,3,4 (mes junio julio agosto septiembre octubre) - y: ventas noviembre
#### Prueba: fechaID= 5 (mes noviembre) - y:ventas diciembre

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulagosepoct_para_predecir_noviembre = mes[4] 
prueba_noviembre_para_predecir_en_diciembre = mes[5] 

print('Entrenar con los meses de Junio, Julio, Agosto, Septiembre y Octubre cuyos índices son:', "0","1","2","3",entrenamiento_junjulagosepoct_para_predecir_noviembre)
print('Probar con el mes de Agosto cuyo índice es:', prueba_noviembre_para_predecir_en_diciembre)

Entrenar con los meses de Junio, Julio, Agosto, Septiembre y Octubre cuyos índices son: 0 1 2 3 4
Probar con el mes de Agosto cuyo índice es: 5


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_5 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulagosepoct_para_predecir_noviembre]
#entrenamiento_5.head(2)
entrenamiento_5.shape

(4342975, 37)

In [12]:
prueba_5 = datos_modelo[datos_modelo['fechaID']==prueba_noviembre_para_predecir_en_diciembre]
#prueba_5.head(2)
prueba_5.shape

(868595, 37)

In [13]:
entrenamiento_5.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_5.head(5)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   
               3                     0                                  0.0   
               4                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   
               3                                       75   
               4                                       77   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   
               3                                  1.041667   
               4                                  1.026667   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   
               3                                     136   
               4                                      86   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   
               3                                 1.079365   
               4                                 1.088608   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   
               3                                   73612   
               4                                   67113   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
               2                                 1.126270   
               3                                 1.110521   
               4                                 1.103143   

                        ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                   
1        1     0                                     1   
               1                                     1   
               2                                     0   
               3                                     0   
               4                                     0   

                        ventas_promedio_por_sku_por_mes  ...  \
tiendaID skuID fechaID                                   ...   
1        1     0                                    1.0  ...   
               1                                    1.0  ...   
               2                                    0.0  ...   
               3                                    0.0  ...   
               4                                    0.0  ...   

                        lag2mes_ventas_totales_por_sk

In [14]:
prueba_5.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_5.head(5)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,5,0,0.0,80,1.126761,45,1.097561,89688,1.182501,0,0.0,...,0,0.0,87,1.060976,264,1.068826,84129,1.12627,0,0.0
2,1,5,0,0.0,11,1.000000,45,1.097561,89688,1.182501,0,0.0,...,0,0.0,13,1.083333,264,1.068826,84129,1.12627,0,0.0
3,1,5,0,0.0,52,1.040000,45,1.097561,89688,1.182501,0,0.0,...,0,0.0,36,1.028571,264,1.068826,84129,1.12627,0,0.0
4,1,5,0,0.0,12,1.000000,45,1.097561,89688,1.182501,0,0.0,...,0,0.0,0,0.000000,264,1.068826,84129,1.12627,0,0.0
5,1,5,0,0.0,17,1.000000,45,1.097561,89688,1.182501,0,0.0,...,0,0.0,0,0.000000,264,1.068826,84129,1.12627,0,0.0


In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_5 = entrenamiento_5.iloc[:, entrenamiento_5.columns != 'variable_y_ventas_del_siguiente_mes']
y_5 = entrenamiento_5.iloc[:, entrenamiento_5.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir en x y y el conjunyo de prueba
prueba_x_5= prueba_5.iloc[:, prueba_5.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_5= prueba_5.iloc[:, prueba_5.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 6. Sexta división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5 (mes junio julio agosto septiembre octubre noviembre) - y: ventas diciembre
#### Prueba: fechaID=6 (mes diciembre) - y:ventas enero

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulagosepoctnov_para_predecir_diciembre = mes[5] 
prueba_diciembre_para_predecir_en_enero = mes[6] 

print('Entrenar con los meses de Junio, Julio, Agosto, Septiembre, Octubre y Noviembre cuyos índices son:', "0","1","2","3","4",entrenamiento_junjulagosepoctnov_para_predecir_diciembre)
print('Probar con el mes de Diciembre cuyo índice es:', prueba_diciembre_para_predecir_en_enero)

Entrenar con los meses de Junio, Julio, Agosto, Septiembre, Octubre y Noviembre cuyos índices son: 0 1 2 3 4 5
Probar con el mes de Diciembre cuyo índice es: 6


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_6 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulagosepoctnov_para_predecir_diciembre]
#entrenamiento_6.head(2)
entrenamiento_6.shape

(5211570, 37)

In [12]:
prueba_6 = datos_modelo[datos_modelo['fechaID']==prueba_diciembre_para_predecir_en_enero]
#prueba_6.head(2)
prueba_6.shape

(868595, 37)

In [13]:
entrenamiento_6.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_6.head(6)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   
               3                     0                                  0.0   
               4                     0                                  0.0   
               5                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   
               3                                       75   
               4                                       77   
               5                                       80   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   
               3                                  1.041667   
               4                                  1.026667   
               5                                  1.126761   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   
               3                                     136   
               4                                      86   
               5                                      45   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   
               3                                 1.079365   
               4                                 1.088608   
               5                                 1.097561   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   
               3                                   73612   
               4                                   67113   
               5                                   89688   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
               2                                 1.126270   
               3                                 1.110521   
               4                                 1.103143   
               5                                 1.182501   

                        ventas_totales_por_sku_por_mes  \
tiendaID skuID fechaID                                   
1        1     0                                     1   
               1                                     1   
               2                                     0   
               3                                     0   
               4                                     0   
               5                                     0   



In [14]:
prueba_6.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_6.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,6,0,0.0,73,1.028169,32,1.066667,95478,1.156594,0,0.0,...,0,0.0,75,1.041667,136,1.079365,73612,1.110521,0,0.0
2,1,6,0,0.0,27,1.038462,32,1.066667,95478,1.156594,0,0.0,...,0,0.0,22,1.047619,136,1.079365,73612,1.110521,0,0.0


In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_6 = entrenamiento_6.iloc[:, entrenamiento_6.columns != 'variable_y_ventas_del_siguiente_mes']
y_6 = entrenamiento_6.iloc[:, entrenamiento_6.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir en x y y el conjunyo de prueba
prueba_x_6= prueba_6.iloc[:, prueba_6.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_6= prueba_6.iloc[:, prueba_6.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 7. Séptima división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5,6 (mes junio julio agosto septiembre octubre noviembre diciembre) - y: ventas enero
#### Prueba: fechaID=7 (mes enero) - y:ventas febrero

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulagosepoctnovdic_para_predecir_enero = mes[6]
prueba_enero_para_predecir_en_febrero = mes[7] 

print('Entrenar con los meses de Junio, Julio, Agosto, Septiembre, Octubre, Noviembre y Diciembre cuyos índices son:', "0","1","2","3","4","5",entrenamiento_junjulagosepoctnovdic_para_predecir_enero)
print('Probar con el mes de Agosto cuyo índice es:', prueba_enero_para_predecir_en_febrero)

Entrenar con los meses de Junio, Julio, Agosto, Septiembre, Octubre, Noviembre y Diciembre cuyos índices son: 0 1 2 3 4 5 6
Probar con el mes de Agosto cuyo índice es: 7


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_7 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulagosepoctnovdic_para_predecir_enero]
#entrenamiento_7.head(2)
entrenamiento_7.shape

(6080165, 37)

In [12]:
prueba_7 = datos_modelo[datos_modelo['fechaID']==prueba_enero_para_predecir_en_febrero]
#prueba_7.head(2)
prueba_7.shape

(868595, 37)

In [13]:
entrenamiento_7.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_7.head(7)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   
               3                     0                                  0.0   
               4                     0                                  0.0   
               5                     0                                  0.0   
               6                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   
               3                                       75   
               4                                       77   
               5                                       80   
               6                                       73   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   
               3                                  1.041667   
               4                                  1.026667   
               5                                  1.126761   
               6                                  1.028169   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   
               3                                     136   
               4                                      86   
               5                                      45   
               6                                      32   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   
               3                                 1.079365   
               4                                 1.088608   
               5                                 1.097561   
               6                                 1.066667   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   
               3                                   73612   
               4                                   67113   
               5                                   89688   
               6                                   95478   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
               1                                 1.137986   
               2                                 1.126270   
               3                                 1.110521   
               4                                 1.103143   
               5                                 1.182501   
               6                                 1.156594   

                     

In [14]:
prueba_7.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_7.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,7,0,0.0,70,1.09375,71,1.145161,66132,1.13079,1,1.0,...,0,0.0,77,1.026667,86,1.088608,67113,1.103143,0,0.0
2,1,7,0,0.0,17,1.00000,71,1.145161,66132,1.13079,1,1.0,...,0,0.0,9,1.125000,86,1.088608,67113,1.103143,0,0.0


In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_7 = entrenamiento_7.iloc[:, entrenamiento_7.columns != 'variable_y_ventas_del_siguiente_mes']
y_7 = entrenamiento_7.iloc[:, entrenamiento_7.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir en x y y el conjunyo de prueba
prueba_x_7= prueba_7.iloc[:, prueba_7.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_7= prueba_7.iloc[:, prueba_7.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 8. Octava división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5,6,7 (mes junio  julio agosto septiembre octubre noviembre diciembre enero) - y: ventas febrero
#### Prueba: fechaID=8(mes febrero) - y:ventas marzo

In [10]:
#Límites para cortar conjuntos en dattos
entrenamiento_junjulagosepoctnovdicene_para_predecir_febrero = mes[7]
prueba_febrero_para_predecir_en_marzo = mes[8] 

print('Entrenar con los meses de Junio,Julio,Agosto,Septiembre,Octubre, Noviembre, Diciembre, y Enero cuyos índices son:', "0","1","2","3","4","5","6",entrenamiento_junjulagosepoctnovdicene_para_predecir_febrero)
print('Probar con el mes de febrero cuyo índice es:', prueba_febrero_para_predecir_en_marzo)

Entrenar con los meses de Junio,Julio,Agosto,Septiembre,Octubre, Noviembre, Diciembre, y Enero cuyos índices son: 0 1 2 3 4 5 6 7
Probar con el mes de febrero cuyo índice es: 8


In [11]:
#dividir el archivo en entrenamiento y prueba
entrenamiento_8 = datos_modelo[datos_modelo['fechaID']<=entrenamiento_junjulagosepoctnovdicene_para_predecir_febrero]
#entrenamiento_8.head(2)
entrenamiento_8.shape


(6948760, 37)

In [12]:
prueba_8 = datos_modelo[datos_modelo['fechaID']==prueba_febrero_para_predecir_en_marzo]
#prueba_8.head(2)
prueba_8.shape

(868595, 37)

In [13]:
entrenamiento_8.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
entrenamiento_8.head(8)

ventas_por_mes  variable_y_ventas_del_siguiente_mes  \
tiendaID skuID fechaID                                                        
1        1     0                     0                                  0.0   
               1                     0                                  0.0   
               2                     0                                  0.0   
               3                     0                                  0.0   
               4                     0                                  0.0   
               5                     0                                  0.0   
               6                     0                                  0.0   
               7                     0                                  0.0   

                        ventas_totales_por_tienda_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                       70   
               1                                       79   
               2                                       87   
               3                                       75   
               4                                       77   
               5                                       80   
               6                                       73   
               7                                       70   

                        ventas_promedio_por_tienda_por_mes  \
tiendaID skuID fechaID                                       
1        1     0                                  1.044776   
               1                                  1.128571   
               2                                  1.060976   
               3                                  1.041667   
               4                                  1.026667   
               5                                  1.126761   
               6                                  1.028169   
               7                                  1.093750   

                        ventas_totales_por_marca_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                     355   
               1                                     379   
               2                                     264   
               3                                     136   
               4                                      86   
               5                                      45   
               6                                      32   
               7                                      71   

                        ventas_promedio_por_marca_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.047198   
               1                                 1.046961   
               2                                 1.068826   
               3                                 1.079365   
               4                                 1.088608   
               5                                 1.097561   
               6                                 1.066667   
               7                                 1.145161   

                        ventas_totales_por_gamma_por_mes  \
tiendaID skuID fechaID                                     
1        1     0                                   86507   
               1                                   92120   
               2                                   84129   
               3                                   73612   
               4                                   67113   
               5                                   89688   
               6                                   95478   
               7                                   66132   

                        ventas_promedio_por_gamma_por_mes  \
tiendaID skuID fechaID                                      
1        1     0                                 1.161542   
     

In [14]:
prueba_8.set_index(['tiendaID', 'skuID','fechaID'], inplace=True)
prueba_8.head(2)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,8,0,0.0,61,1.070175,53,1.081633,65906,1.125809,4,1.0,...,0,0.0,80,1.126761,45,1.097561,89688,1.182501,0,0.0
2,1,8,0,0.0,15,1.000000,53,1.081633,65906,1.125809,4,1.0,...,0,0.0,11,1.000000,45,1.097561,89688,1.182501,0,0.0


In [15]:
#Dividir x y y del conjunto de entrenamiento 
x_8 = entrenamiento_8.iloc[:, entrenamiento_8.columns != 'variable_y_ventas_del_siguiente_mes']
y_8 = entrenamiento_8.iloc[:, entrenamiento_8.columns == 'variable_y_ventas_del_siguiente_mes']

In [16]:
#Dividir n x y y el conjunyo de prueba
prueba_x_8= prueba_8.iloc[:, prueba_8.columns != 'variable_y_ventas_del_siguiente_mes']
prueba_y_8= prueba_8.iloc[:, prueba_8.columns == 'variable_y_ventas_del_siguiente_mes']

# <font color='darkblue'> --------------------.--------------------.--------------------.--------------------.--------------------.--------------------. </font>

# <font color='blue'> C. ENTRENAMIENTO (8 conjuntos de entrenamiento - 8 modelos) </font>

In [17]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
from scipy.stats import sem
#from sklearn.model_selection import TimeSeriesSplit
import warnings; warnings.simplefilter('ignore')

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 1. Primera división: 
#### Entrenamiento: fechaID=0 (mes junio) - y: ventas julio

In [18]:
# Fit regression model
regr_1 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_1.fit(x_1, y_1) #5:04 - 05:08

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_1_entrenamiento = regr_1.predict(x_1) #menos de 1 minuto

In [21]:
max(pred_1_entrenamiento)

56.93

In [22]:
min(pred_1_entrenamiento)

0.0

In [23]:
pred_1_entrenamiento = np.clip(pred_1_entrenamiento,0,200)
prediccion_1_entrenamiento = pd.DataFrame(pred_1_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_1_entrenamiento = prediccion_1_entrenamiento.round()

In [24]:
prediccion_1_entrenamiento.sum()


ventas_por_mes_pred    104965.0
dtype: float64

In [25]:
prediccion_1_entrenamiento = prediccion_1_entrenamiento.values
prediccion_1_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_1['ventas_por_mes_pred'] = prediccion_1_entrenamiento

In [27]:
entrenamiento_1.head(5)

,,,ventas_por_mes,variable_y_ventas_del_siguiente_mes,ventas_totales_por_tienda_por_mes,ventas_promedio_por_tienda_por_mes,ventas_totales_por_marca_por_mes,ventas_promedio_por_marca_por_mes,ventas_totales_por_gamma_por_mes,ventas_promedio_por_gamma_por_mes,ventas_totales_por_sku_por_mes,ventas_promedio_por_sku_por_mes,...,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes,ventas_por_mes_pred
tiendaID,skuID,fechaID,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0,...,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0,0.0
2,1,0,0,0.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0,...,1.0,14,1.076923,355,1.047198,86507,1.161542,1,1.0,0.0
3,1,0,0,0.0,24,1.000000,355,1.047198,86507,1.161542,1,1.0,...,1.0,24,1.000000,355,1.047198,86507,1.161542,1,1.0,0.0
4,1,0,0,0.0,0,0.000000,355,1.047198,86507,1.161542,1,1.0,...,1.0,0,0.000000,355,1.047198,86507,1.161542,1,1.0,0.0
5,1,0,0,0.0,0,0.000000,355,1.047198,86507,1.161542,1,1.0,...,1.0,0,0.000000,355,1.047198,86507,1.161542,1,1.0,0.0


In [28]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_1_ent= mean_absolute_error(entrenamiento_1.variable_y_ventas_del_siguiente_mes, entrenamiento_1.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_1_ent = mean_squared_error(entrenamiento_1.variable_y_ventas_del_siguiente_mes, entrenamiento_1.ventas_por_mes_pred)

# R-squared scores
r2_1_ent = r2_score(entrenamiento_1.variable_y_ventas_del_siguiente_mes, entrenamiento_1.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_1_ent, 6))
print('Mean Squared Error:', round(mse_1_ent, 6))
print('R-squared scores:', round(r2_1_ent, 6))


Mean Absolute Error: 0.059521
Mean Squared Error: 0.118421
R-squared scores: 0.856324


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 2. Segunda división: 
#### Entrenamiento: fechaID=0 y 1 (mes junio y julio) - y: ventas Agosto


In [44]:
# Fit regression model
regr_2 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [45]:
regr_2.fit(x_2, y_2) #05:10 - 5:20

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [46]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_2_entrenamiento = regr_2.predict(x_2) #menos de 1 min

In [47]:
max(pred_2_entrenamiento)

79.19

In [48]:
min(pred_2_entrenamiento)

0.0

In [49]:
pred_2_entrenamiento = np.clip(pred_2_entrenamiento,0,200)
prediccion_2_entrenamiento = pd.DataFrame(pred_2_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_2_entrenamiento = prediccion_2_entrenamiento.round()

In [50]:
prediccion_2_entrenamiento.sum()

ventas_por_mes_pred    213285.0
dtype: float64

In [51]:
prediccion_2_entrenamiento = prediccion_2_entrenamiento.values
prediccion_2_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [52]:
entrenamiento_2['ventas_por_mes_pred'] = prediccion_2_entrenamiento

In [53]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_2_ent= mean_absolute_error(entrenamiento_2.variable_y_ventas_del_siguiente_mes, entrenamiento_2.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_2_ent = mean_squared_error(entrenamiento_2.variable_y_ventas_del_siguiente_mes, entrenamiento_2.ventas_por_mes_pred)

# R-squared scores
r2_2_ent = r2_score(entrenamiento_2.variable_y_ventas_del_siguiente_mes, entrenamiento_2.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_2_ent, 6))
print('Mean Squared Error:', round(mse_2_ent, 6))
print('R-squared scores:', round(r2_2_ent, 6))


Mean Absolute Error: 0.04449
Mean Squared Error: 0.083847
R-squared scores: 0.897438


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 3. Tercera división: 
#### Entrenamiento: fechaID=0,1,2(mes junio julio agosto) - y: ventas septiembre


In [69]:
# Fit regression model
regr_3 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [70]:
regr_3.fit(x_3, y_3) #05:23 - 5:43

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [71]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_3_entrenamiento = regr_3.predict(x_3) #menos de 1 min

In [72]:
max(pred_3_entrenamiento)

73.84

In [73]:
min(pred_3_entrenamiento)

0.0

In [74]:
pred_3_entrenamiento = np.clip(pred_3_entrenamiento,0,200)
prediccion_3_entrenamiento = pd.DataFrame(pred_3_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_3_entrenamiento = prediccion_3_entrenamiento.round()

In [75]:
prediccion_3_entrenamiento.sum()

ventas_por_mes_pred    305854.0
dtype: float64

In [76]:
prediccion_3_entrenamiento = prediccion_3_entrenamiento.values
prediccion_3_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [77]:
entrenamiento_3['ventas_por_mes_pred'] = prediccion_3_entrenamiento

In [78]:
# Mean absolute error (MAE)
mae_3_ent= mean_absolute_error(entrenamiento_3.variable_y_ventas_del_siguiente_mes, entrenamiento_3.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_3_ent = mean_squared_error(entrenamiento_3.variable_y_ventas_del_siguiente_mes, entrenamiento_3.ventas_por_mes_pred)

# R-squared scores
r2_3_ent = r2_score(entrenamiento_3.variable_y_ventas_del_siguiente_mes, entrenamiento_3.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_3_ent, 6))
print('Mean Squared Error:', round(mse_3_ent, 6))
print('R-squared scores:', round(r2_3_ent, 6))


Mean Absolute Error: 0.039514
Mean Squared Error: 0.071958
R-squared scores: 0.903418


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 4. Cuarta división: 
#### Entrenamiento: fechaID=0,1,2,3 (mes junio julio agosto septiembre) - y: ventas octubre

In [18]:
# Fit regression model
regr_4 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_4.fit(x_4, y_4) #5:48 - 6:16

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_4_entrenamiento = regr_4.predict(x_4) #menos de 1 minuto

In [21]:
max(pred_4_entrenamiento)

78.49

In [22]:
min(pred_4_entrenamiento)

0.0

In [23]:
pred_4_entrenamiento = np.clip(pred_4_entrenamiento,0,200)
prediccion_4_entrenamiento = pd.DataFrame(pred_4_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_4_entrenamiento = prediccion_4_entrenamiento.round()

In [24]:
prediccion_4_entrenamiento.sum()

ventas_por_mes_pred    402383.0
dtype: float64

In [25]:
prediccion_4_entrenamiento = prediccion_4_entrenamiento.values
prediccion_4_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_4['ventas_por_mes_pred'] = prediccion_4_entrenamiento

In [28]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_4_ent= mean_absolute_error(entrenamiento_4.variable_y_ventas_del_siguiente_mes, entrenamiento_4.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_4_ent = mean_squared_error(entrenamiento_4.variable_y_ventas_del_siguiente_mes, entrenamiento_4.ventas_por_mes_pred)

# R-squared scores
r2_4_ent = r2_score(entrenamiento_4.variable_y_ventas_del_siguiente_mes, entrenamiento_4.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_4_ent, 6))
print('Mean Squared Error:', round(mse_4_ent, 6))
print('R-squared scores:', round(r2_4_ent, 6))


Mean Absolute Error: 0.035636
Mean Squared Error: 0.062938
R-squared scores: 0.91058


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 5. Quinta división: 
#### Entrenamiento: fechaID=0,1,2,3,4 (mes junio julio agosto septiembre octubre) - y: ventas noviembre


In [18]:
# Fit regression model
regr_5 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_5.fit(x_5, y_5) #7:01

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_5_entrenamiento = regr_5.predict(x_5) #menos de 1 minuto

In [21]:
pred_5_entrenamiento.shape

(4342975,)

In [22]:
max(pred_5_entrenamiento)

75.49

In [23]:
min(pred_5_entrenamiento)

0.0

In [24]:
pred_5_entrenamiento = np.clip(pred_5_entrenamiento,0,200)
prediccion_5_entrenamiento = pd.DataFrame(pred_5_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_5_entrenamiento = prediccion_5_entrenamiento.round()

In [25]:
prediccion_5_entrenamiento = prediccion_5_entrenamiento.values
prediccion_5_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_5['ventas_por_mes_pred'] = prediccion_5_entrenamiento

In [27]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_5_ent= mean_absolute_error(entrenamiento_5.variable_y_ventas_del_siguiente_mes, entrenamiento_5.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_5_ent = mean_squared_error(entrenamiento_5.variable_y_ventas_del_siguiente_mes, entrenamiento_5.ventas_por_mes_pred)

# R-squared scores
r2_5_ent = r2_score(entrenamiento_5.variable_y_ventas_del_siguiente_mes, entrenamiento_5.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_5_ent, 6))
print('Mean Squared Error:', round(mse_5_ent, 6))
print('R-squared scores:', round(r2_5_ent, 6))


Mean Absolute Error: 0.037678
Mean Squared Error: 0.070332
R-squared scores: 0.902626


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 6. Sexta división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5 (mes junio julio agosto septiembre octubre noviembre) - y: ventas diciembre

In [18]:
# Fit regression model
regr_6 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_6.fit(x_6, y_6) #8:21 - 9:16

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_6_entrenamiento = regr_6.predict(x_6) #menos de 1 minuto

In [21]:
pred_6_entrenamiento.shape

(5211570,)

In [22]:
max(pred_6_entrenamiento)

71.7

In [23]:
min(pred_6_entrenamiento)

0.0

In [24]:
pred_6_entrenamiento = np.clip(pred_6_entrenamiento,0,200)
prediccion_6_entrenamiento = pd.DataFrame(pred_6_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_6_entrenamiento = prediccion_6_entrenamiento.round()

In [25]:
prediccion_6_entrenamiento = prediccion_6_entrenamiento.values
prediccion_6_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_6['ventas_por_mes_pred'] = prediccion_6_entrenamiento

In [27]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_6_ent= mean_absolute_error(entrenamiento_6.variable_y_ventas_del_siguiente_mes, entrenamiento_6.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_6_ent = mean_squared_error(entrenamiento_6.variable_y_ventas_del_siguiente_mes, entrenamiento_6.ventas_por_mes_pred)

# R-squared scores
r2_6_ent = r2_score(entrenamiento_6.variable_y_ventas_del_siguiente_mes, entrenamiento_6.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_6_ent, 6))
print('Mean Squared Error:', round(mse_6_ent, 6))
print('R-squared scores:', round(r2_6_ent, 6))


Mean Absolute Error: 0.037277
Mean Squared Error: 0.068222
R-squared scores: 0.909142


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 7. Séptima división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5,6 (mes junio julio agosto septiembre octubre noviembre diciembre) - y: ventas enero


In [18]:
# Fit regression model
regr_7 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_7.fit(x_7, y_7) #9:25-10_27

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_7_entrenamiento = regr_7.predict(x_7) #menos de 1 minuto

In [21]:
pred_7_entrenamiento.shape

(6080165,)

In [22]:
max(pred_7_entrenamiento)

74.62

In [23]:
min(pred_7_entrenamiento)

0.0

In [24]:
pred_7_entrenamiento = np.clip(pred_7_entrenamiento,0,200)
prediccion_7_entrenamiento = pd.DataFrame(pred_7_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_7_entrenamiento = prediccion_7_entrenamiento.round()

In [25]:
prediccion_7_entrenamiento = prediccion_7_entrenamiento.values
prediccion_7_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_7['ventas_por_mes_pred'] = prediccion_7_entrenamiento

In [27]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_7_ent= mean_absolute_error(entrenamiento_7.variable_y_ventas_del_siguiente_mes, entrenamiento_7.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_7_ent = mean_squared_error(entrenamiento_7.variable_y_ventas_del_siguiente_mes, entrenamiento_7.ventas_por_mes_pred)

# R-squared scores
r2_7_ent = r2_score(entrenamiento_7.variable_y_ventas_del_siguiente_mes, entrenamiento_7.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_7_ent, 6))
print('Mean Squared Error:', round(mse_7_ent, 6))
print('R-squared scores:', round(r2_7_ent, 6))

Mean Absolute Error: 0.036986
Mean Squared Error: 0.068593
R-squared scores: 0.904661


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 8. Octava división: 
#### Entrenamiento: fechaID=0,1,2,3,4,5,6,7 (mes junio  julio agosto septiembre octubre noviembre diciembre enero) - y: ventas febrero


In [18]:
# Fit regression model
regr_8 = RandomForestRegressor(
    n_estimators=100, 
    n_jobs=-1)

In [19]:
regr_8.fit(x_8, y_8) #12:31-1:50

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
#predicciones con conjunto de entrenamiento para calcular el error de entrenamiento
pred_8_entrenamiento = regr_8.predict(x_8) #menos de 1 minuto

In [21]:
pred_8_entrenamiento.shape

(6948760,)

In [22]:
max(pred_8_entrenamiento)

70.55

In [23]:
min(pred_8_entrenamiento)

0.0

In [24]:
pred_8_entrenamiento = np.clip(pred_8_entrenamiento,0,200)
prediccion_8_entrenamiento = pd.DataFrame(pred_8_entrenamiento, columns = ['ventas_por_mes_pred'])
prediccion_8_entrenamiento = prediccion_8_entrenamiento.round()

In [25]:
prediccion_8_entrenamiento = prediccion_8_entrenamiento.values
prediccion_8_entrenamiento

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
entrenamiento_8['ventas_por_mes_pred'] = prediccion_8_entrenamiento

In [27]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae_8_ent= mean_absolute_error(entrenamiento_8.variable_y_ventas_del_siguiente_mes, entrenamiento_8.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_8_ent = mean_squared_error(entrenamiento_8.variable_y_ventas_del_siguiente_mes, entrenamiento_8.ventas_por_mes_pred)

# R-squared scores
r2_8_ent = r2_score(entrenamiento_8.variable_y_ventas_del_siguiente_mes, entrenamiento_8.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_8_ent, 6))
print('Mean Squared Error:', round(mse_8_ent, 6))
print('R-squared scores:', round(r2_8_ent, 6))

Mean Absolute Error: 0.035426
Mean Squared Error: 0.065379
R-squared scores: 0.906369


# <font color='darkblue'> --------------------.--------------------.--------------------.--------------------.--------------------.--------------------. </font>

# <font color='blue'> D. PRUEBA (8 conjuntos de prueba - 8 modelos) </font>

# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 1. Primera división: 
#### Prueba: fechaID=1 (mes julio) - y:ventas agosto

In [29]:
#predicciones
pred_1 = regr_1.predict(prueba_x_1) #1 min

In [30]:
max(pred_1)

56.48

In [31]:
min(pred_1)

0.0

In [32]:
pred_1 = np.clip(pred_1,0,200)
prediccion_1 = pd.DataFrame(pred_1, columns = ['ventas_por_mes_pred'])
prediccion_1 = prediccion_1.round()

In [33]:
prediccion_1.sum()

ventas_por_mes_pred    123362.0
dtype: float64

In [34]:
prediccion_1 = prediccion_1.values

In [35]:
prueba_1['ventas_por_mes_pred'] = prediccion_1

In [36]:
# Mean absolute error (MAE)
mae_1_prueba= mean_absolute_error(prueba_1.variable_y_ventas_del_siguiente_mes, prueba_1.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_1_prueba = mean_squared_error(prueba_1.variable_y_ventas_del_siguiente_mes, prueba_1.ventas_por_mes_pred)

# R-squared scores
r2_1_prueba = r2_score(prueba_1.variable_y_ventas_del_siguiente_mes, prueba_1.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_1_prueba, 6))
print('Mean Squared Error:', round(mse_1_prueba, 6))
print('R-squared scores:', round(r2_1_prueba, 6))


Mean Absolute Error: 0.134817
Mean Squared Error: 0.457043
R-squared scores: 0.43631


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 2. Segunda división: 
#### Prueba: fechaID=2 (mes Agosto) - y:ventas septiembre

In [54]:
#predicciones
pred_2 = regr_2.predict(prueba_x_2) #5:21

In [55]:
max(pred_2)

68.62

In [56]:
min(pred_2)

0.0

In [57]:
pred_2 = np.clip(pred_2,0,200)
prediccion_2 = pd.DataFrame(pred_2, columns = ['ventas_por_mes_pred'])
prediccion_2 = prediccion_2.round()

In [58]:
prediccion_2.sum()

ventas_por_mes_pred    120702.0
dtype: float64

In [59]:
prediccion_2 = prediccion_2.values

In [60]:
prueba_2['ventas_por_mes_pred'] = prediccion_2

In [61]:
# Mean absolute error (MAE)
mae_2_prueba= mean_absolute_error(prueba_2.variable_y_ventas_del_siguiente_mes, prueba_2.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_2_prueba = mean_squared_error(prueba_2.variable_y_ventas_del_siguiente_mes, prueba_2.ventas_por_mes_pred)

# R-squared scores
r2_2_prueba = r2_score(prueba_2.variable_y_ventas_del_siguiente_mes, prueba_2.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_2_prueba, 6))
print('Mean Squared Error:', round(mse_2_prueba, 6))
print('R-squared scores:', round(r2_2_prueba, 6))


Mean Absolute Error: 0.121601
Mean Squared Error: 0.332171
R-squared scores: 0.446231


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 3. Tercera división: 
#### Prueba: fechaID=3 (mes septiembre) - y:ventas octubre

In [79]:
#predicciones
pred_3 = regr_3.predict(prueba_x_3) #10:33

In [80]:
max(pred_3)

66.72

In [81]:
min(pred_3)

0.0

In [82]:
pred_3 = np.clip(pred_3,0,200)
prediccion_3 = pd.DataFrame(pred_3, columns = ['ventas_por_mes_pred'])
prediccion_3 = prediccion_3.round()

In [83]:
prediccion_3.sum()

ventas_por_mes_pred    98292.0
dtype: float64

In [84]:
prediccion_3 = prediccion_3.values

In [86]:
prueba_3['ventas_por_mes_pred'] = prediccion_3

In [87]:
# Mean absolute error (MAE)
mae_3_prueba= mean_absolute_error(prueba_3.variable_y_ventas_del_siguiente_mes, prueba_3.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_3_prueba = mean_squared_error(prueba_3.variable_y_ventas_del_siguiente_mes, prueba_3.ventas_por_mes_pred)

# R-squared scores
r2_3_prueba = r2_score(prueba_3.variable_y_ventas_del_siguiente_mes, prueba_3.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_3_prueba, 6))
print('Mean Squared Error:', round(mse_3_prueba, 6))
print('R-squared scores:', round(r2_3_prueba, 6))


Mean Absolute Error: 0.120359
Mean Squared Error: 0.375159
R-squared scores: 0.353355


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 4. Cuarta división: 
#### Prueba: fechaID=4 (mes octubre) - y:ventas noviembre

In [29]:
#predicciones
pred_4 = regr_4.predict(prueba_x_4) #10:42

In [30]:
max(pred_4)

36.09

In [31]:
min(pred_4)

0.0

In [32]:
pred_4 = np.clip(pred_4,0,200)
prediccion_4 = pd.DataFrame(pred_4, columns = ['ventas_por_mes_pred'])
prediccion_4 = prediccion_4.round()

In [33]:
prediccion_4.sum()

ventas_por_mes_pred    87556.0
dtype: float64

In [34]:
prediccion_4 = prediccion_4.values

In [35]:
prueba_4['ventas_por_mes_pred'] = prediccion_4

In [36]:
# Mean absolute error (MAE)
mae_4_prueba= mean_absolute_error(prueba_4.variable_y_ventas_del_siguiente_mes, prueba_4.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_4_prueba = mean_squared_error(prueba_4.variable_y_ventas_del_siguiente_mes, prueba_4.ventas_por_mes_pred)

# R-squared scores
r2_4_prueba = r2_score(prueba_4.variable_y_ventas_del_siguiente_mes, prueba_4.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_4_prueba, 6))
print('Mean Squared Error:', round(mse_4_prueba, 6))
print('R-squared scores:', round(r2_4_prueba, 6))


Mean Absolute Error: 0.134992
Mean Squared Error: 0.488139
R-squared scores: 0.386528


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 5. Quinta división: 

#### Prueba: fechaID= 5 (mes noviembre) - y:ventas diciembre

In [28]:
#predicciones
pred_5 = regr_5.predict(prueba_x_5) #11:02

In [29]:
max(pred_5)

54.01

In [30]:
min(pred_5)

0.0

In [31]:
pred_5 = np.clip(pred_5,0,200)
prediccion_5 = pd.DataFrame(pred_5, columns = ['ventas_por_mes_pred'])
prediccion_5 = prediccion_5.round()

In [32]:
prediccion_5.sum()

ventas_por_mes_pred    185403.0
dtype: float64

In [33]:
prediccion_5 = prediccion_5.values

In [34]:
prueba_5['ventas_por_mes_pred'] = prediccion_5

In [35]:
# Mean absolute error (MAE)
mae_5_prueba= mean_absolute_error(prueba_5.variable_y_ventas_del_siguiente_mes, prueba_5.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_5_prueba = mean_squared_error(prueba_5.variable_y_ventas_del_siguiente_mes, prueba_5.ventas_por_mes_pred)

# R-squared scores
r2_5_prueba = r2_score(prueba_5.variable_y_ventas_del_siguiente_mes, prueba_5.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_5_prueba, 6))
print('Mean Squared Error:', round(mse_5_prueba, 6))
print('R-squared scores:', round(r2_5_prueba, 6))

Mean Absolute Error: 0.213052
Mean Squared Error: 0.892317
R-squared scores: 0.000879


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 6. Sexta división: 
#### Prueba: fechaID=6 (mes diciembre) - y:ventas enero

In [28]:
#predicciones
pred_6 = regr_6.predict(prueba_x_6) #11:02

In [29]:
max(pred_6)

36.13

In [30]:
min(pred_6)

0.0

In [31]:
pred_6 = np.clip(pred_6,0,200)
prediccion_6 = pd.DataFrame(pred_6, columns = ['ventas_por_mes_pred'])
prediccion_6 = prediccion_6.round()

In [32]:
prediccion_6.sum()

ventas_por_mes_pred    154964.0
dtype: float64

In [33]:
prediccion_6 = prediccion_6.values

In [34]:
prueba_6['ventas_por_mes_pred'] = prediccion_6

In [35]:
# Mean absolute error (MAE)
mae_6_prueba= mean_absolute_error(prueba_6.variable_y_ventas_del_siguiente_mes, prueba_6.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_6_prueba = mean_squared_error(prueba_6.variable_y_ventas_del_siguiente_mes, prueba_6.ventas_por_mes_pred)

# R-squared scores
r2_6_prueba = r2_score(prueba_6.variable_y_ventas_del_siguiente_mes, prueba_6.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_6_prueba, 6))
print('Mean Squared Error:', round(mse_6_prueba, 6))
print('R-squared scores:', round(r2_6_prueba, 6))

Mean Absolute Error: 0.171851
Mean Squared Error: 0.573246
R-squared scores: -0.08038


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 7. Séptima división: 

#### Prueba: fechaID=7 (mes enero) - y:ventas febrero

In [28]:
#predicciones
pred_7 = regr_7.predict(prueba_x_7) #11:02

In [29]:
max(pred_7)

33.86

In [30]:
min(pred_7)

0.0

In [31]:
pred_7 = np.clip(pred_7,0,200)
prediccion_7 = pd.DataFrame(pred_7, columns = ['ventas_por_mes_pred'])
prediccion_7 = prediccion_7.round()

In [32]:
prediccion_7.sum()

ventas_por_mes_pred    96863.0
dtype: float64

In [33]:
prediccion_7 = prediccion_7.values

In [34]:
prueba_7['ventas_por_mes_pred'] = prediccion_7

In [35]:
# Mean absolute error (MAE)
mae_7_prueba= mean_absolute_error(prueba_7.variable_y_ventas_del_siguiente_mes, prueba_7.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_7_prueba = mean_squared_error(prueba_7.variable_y_ventas_del_siguiente_mes, prueba_7.ventas_por_mes_pred)

# R-squared scores
r2_7_prueba = r2_score(prueba_7.variable_y_ventas_del_siguiente_mes, prueba_7.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_7_prueba, 6))
print('Mean Squared Error:', round(mse_7_prueba, 6))
print('R-squared scores:', round(r2_7_prueba, 6))

Mean Absolute Error: 0.120704
Mean Squared Error: 0.361442
R-squared scores: 0.341974


# <font color='darkblue'>-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-..-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.- </font>

## 8. Octava división: 
#### Prueba: fechaID=8(mes febrero) - y:ventas marzo

In [28]:
#predicciones
pred_8 = regr_8.predict(prueba_x_8) #11:02

In [29]:
max(pred_8)

46.58

In [30]:
min(pred_8)

0.0

In [31]:
pred_8 = np.clip(pred_8,0,200)
prediccion_8 = pd.DataFrame(pred_8, columns = ['ventas_por_mes_pred'])
prediccion_8 = prediccion_8.round()

In [32]:
prediccion_8.sum()

ventas_por_mes_pred    94530.0
dtype: float64

In [33]:
prediccion_8 = prediccion_8.values

In [34]:
prueba_8['ventas_por_mes_pred'] = prediccion_8

In [35]:
# Mean absolute error (MAE)
mae_8_prueba= mean_absolute_error(prueba_8.variable_y_ventas_del_siguiente_mes, prueba_8.ventas_por_mes_pred)

# Mean squared error (MSE)
mse_8_prueba = mean_squared_error(prueba_8.variable_y_ventas_del_siguiente_mes, prueba_8.ventas_por_mes_pred)

# R-squared scores
r2_8_prueba = r2_score(prueba_8.variable_y_ventas_del_siguiente_mes, prueba_8.ventas_por_mes_pred)


# Print metrics
print('Mean Absolute Error:', round(mae_8_prueba, 6))
print('Mean Squared Error:', round(mse_8_prueba, 6))
print('R-squared scores:', round(r2_8_prueba, 6))

Mean Absolute Error: 0.106045
Mean Squared Error: 0.308035
R-squared scores: -0.166723
